# Exam Problem 

Cristina Hernández, Beatriz Jimenez, Macarena Vargas, Guillermo Ruiz


In [41]:
import pyomo.environ as pe
import pyomo.opt as po

#### Create the model 

In [42]:
model = pe.ConcreteModel()

#### Sets


${n}$ : Number of days = {1, 2, 3 , 4}

${c}$ : Courses = {A, B, C, D}

In [43]:
model.courses = pe.Set(initialize=['A', 'B', 'C', 'D'])
model.days = pe.Set(initialize=[1, 2, 3, 4])

#### Parameters

${G_{n, c}}$ : grade in course c per number of days n 

In [44]:
dict_grades_per_days =  {
    ('A', 1): 3, 
    ('A', 2): 5, 
    ('A', 3): 6, 
    ('A', 4): 7,

    ('B', 1): 4, 
    ('B', 2): 5, 
    ('B', 3): 6, 
    ('B', 4): 9, 

    ('C', 1): 2, 
    ('C', 2): 5, 
    ('C', 3): 7, 
    ('C', 4): 8, 
    
    ('D', 1): 5, 
    ('D', 2): 6, 
    ('D', 3): 7, 
    ('D', 4): 9}

# model.grades_per_days = pe.Param(model.courses, model.days, initialize = dict_grades_per_days)


# Parámetro G[n,c]
model.grades_per_days = pe.Param(model.courses, model.days,
                                 initialize={(c, n): dict_grades_per_days[(c, n)] 
                                             for c in model.courses for n in model.days})

# Parámetro Gmin[c]
dict_gmin = {c: min(dict_grades_per_days[(c, n)] for n in model.days) for c in model.courses}
model.Gmin = pe.Param(model.courses, initialize=dict_gmin)

${w}$ : days to study

In [45]:
model.W = pe.Param(initialize=7)
model.ord = pe.Param(model.days, initialize={n: n for n in model.days})


#### Variables 

${x_{n, c}}$:  number of days n to course c {0,1}

In [46]:
model.days_per_course = pe.Var(model.courses, model.days,  within=pe.Binary)


${y_{c}}$: fail or pass exam {0,1}

In [47]:
model.pass_course = pe.Var(model.courses,within = pe.Binary)

#### Constrains 

Constraint #1: The student wants to dedicate at least one
day to each of the four courses

$\sum_n x_{n, c} = 1 \quad \forall c$

In [48]:
def one_day_per_course(model, c):
    return sum(model.days_per_course[c, n] for n in model.days) == 1

model.one_day_per_course = pe.Constraint(model.courses, rule=one_day_per_course)



Constraint #2: The student only has seven days to study

$\sum_{n,c} ord(n) * x_{n,c} \leq 7$


In [49]:
def total_days(model):
    return sum(model.ord[n] * model.days_per_course[c, n] for c in model.courses for n in model.days) <= model.W

model.total_days = pe.Constraint(rule=total_days)

Constraint #3: Passing condition for each course

For each course 𝑐, we introduce a binary variable $\ y_c$ that indicates whether the course is passed. 

If $\ y_c$ = 1, the grade obtained must be at least 5.

If $\ y_c$ = 0, the constraint is inactive (it only requires the minimum grade possible, which is always satisfied)

$\sum_{n} G_{n,c} \cdot x_{c,n} \;\;\ge\;\; G^{min}_c + (5 - G^{min}_c) \cdot y_c \quad \forall c$


In [50]:
# Variable binaria: y_c = 1 si el curso c está aprobado
model.pass_exam = pe.Var(model.courses, domain=pe.Binary)

# Restricción: condición de aprobado
def pass_condition_rule(model, c):
    return sum(model.grades_per_days[c, n] * model.days_per_course[c, n] for n in model.days) \
           >= model.Gmin[c] + (5 - model.Gmin[c]) * model.pass_exam[c]

model.pass_condition = pe.Constraint(model.courses, rule=pass_condition_rule)

#### Solving the problem  

##### a) Optimal studying schedule to pass the maximum number of exams

##### Objetive function --> Maximize passed exams: max $\sum_c y_{c}$

In [51]:
def obj_passed_exams(model):
    return sum(model.pass_exam[c] for c in model.courses)

model.obj_passed = pe.Objective(rule=obj_passed_exams, sense=pe.maximize)

In [52]:
# Activamos la función objetivo del apartdo a
model.obj_passed.activate()

In [53]:
solver = pe.SolverFactory('gurobi')
result = solver.solve(model, tee=True)

print("Status:", result.solver.status)
print("Termination:", result.solver.termination_condition)

Set parameter Username
Set parameter LicenseID to value 2707272
Academic license - for non-commercial use only - expires 2026-09-11
Read LP format model from file C:\Users\crist\AppData\Local\Temp\tmp3bfqw31r.pyomo.lp
Reading time = 0.00 seconds
x1: 9 rows, 20 columns, 51 nonzeros
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-1355U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 9 rows, 20 columns and 51 nonzeros
Model fingerprint: 0xbfcff691
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 9 rows and 20 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex i

In [54]:
print("\n--- Optimal studying schedule (maximize passed exams) ---")

# Días, nota y aprobado por curso
for c in model.courses:
    for n in model.days:
        if pe.value(model.days_per_course[c, n]) > 0.5:   # variable binaria activa
            grade = pe.value(model.grades_per_days[c, n])
            passed = int(round(pe.value(model.pass_exam[c])))
            print(f"Course {c}: {n} days  |  grade = {grade:.1f}  |  passed = {passed}")

# Total de aprobados
num_passed = sum(int(round(pe.value(model.pass_exam[c]))) for c in model.courses)
print(f"\nTotal passed exams: {num_passed} / {len(model.courses)}")



--- Optimal studying schedule (maximize passed exams) ---
Course A: 2 days  |  grade = 5.0  |  passed = 1
Course B: 2 days  |  grade = 5.0  |  passed = 1
Course C: 2 days  |  grade = 5.0  |  passed = 1
Course D: 1 days  |  grade = 5.0  |  passed = 1

Total passed exams: 4 / 4


In [55]:
# Desactivamos la función objetivo del apartado a
model.obj_passed.deactivate()

##### b) Optimal studying schedule to maximize the average grade in all exams

##### Objetive function --> Maximize average grade in all exams: max $\sum_{n,c} \frac{G_{n,c} * X_{n,c}}{4}$

In [56]:
def obj_average_grade(model):
    return (1/len(model.courses)) * sum(
        model.grades_per_days[c, n] * model.days_per_course[c, n]
        for c in model.courses for n in model.days
    )

model.obj_average = pe.Objective(rule=obj_average_grade, sense=pe.maximize)

In [57]:
solver = pe.SolverFactory('gurobi')
result = solver.solve(model, tee=True)

print("Status:", result.solver.status)
print("Termination:", result.solver.termination_condition)

Set parameter Username
Set parameter LicenseID to value 2707272
Academic license - for non-commercial use only - expires 2026-09-11
Read LP format model from file C:\Users\crist\AppData\Local\Temp\tmpr99g6nb4.pyomo.lp
Reading time = 0.00 seconds
x1: 9 rows, 19 columns, 51 nonzeros
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-1355U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 9 rows, 19 columns and 51 nonzeros
Model fingerprint: 0xa8581318
Variable types: 0 continuous, 19 integer (19 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [5e-01, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+00]
Found heuristic solution: objective 4.5000000
Presolve removed 4 rows and 3 columns
Presolve time: 0.00s
Presolved: 5 rows, 16 columns, 28 nonzeros
Variable types: 0 continuou

In [58]:
# Días asignados y notas por curso
print("\n--- Optimal studying schedule (maximize average grade) ---")
for c in model.courses:
    for n in model.days:
        if pe.value(model.days_per_course[c, n]) > 0.5:   # variable binaria activa
            grade = pe.value(model.grades_per_days[c, n])
            print(f"Course {c}: {n} days  |  grade = {grade}")

# Nota media
average_grade = sum(
    pe.value(model.grades_per_days[c, n]) * pe.value(model.days_per_course[c, n])
    for c in model.courses for n in model.days
) / len(model.courses)

print(f"\nAverage grade: {average_grade:.2f}")


--- Optimal studying schedule (maximize average grade) ---
Course A: 2 days  |  grade = 5
Course B: 1 days  |  grade = 4
Course C: 3 days  |  grade = 7
Course D: 1 days  |  grade = 5

Average grade: 5.25
